In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import spectral
from sklearn import preprocessing
from sklearn.cluster import KMeans 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
#from xgboost import XGBClassifier
from sklearn.decomposition import PCA
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.ensemble import AdaBoostClassifier
#import lightgbm as lgb
import joblib

##  相关函数定义

In [83]:
## 降维操作
def pca_hpi(input_image, dimension=30):
    spectral = list()
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            spectral.append(input_image[i][j,])
    spectral_array = np.array(spectral)
    ## 降维
    pca = PCA(n_components=dimension,svd_solver='auto')
    pca.fit(spectral_array)
    pca_input_iamge = pca.transform(spectral_array)
    ## 姜维之后恢复 原来形状
    pca_return_array = np.vsplit(pca_input_iamge,145)
    pca_return_array = np.array(pca_return_array)
    print (pca.explained_variance_ratio_)
    return pca_return_array

#####-------------------------------------------------- 生成 3D Tensor---------------------------########

# 可以提取出高光谱数据的一个channel的数据 145*145
def split_one_channel(A,channel):
    row = A.shape[0]
    d = [0]*row
    for i in range(row):
        d[i] = A[i][:,channel-1]
    e = np.vstack((d[i] for i in range(row)))
    return e   
# 参数说明
# channel 通道
# A 高光谱立方块


# 依据patch大小（Filter 大小）对矩阵进行镜像处理
def mirror_matrix(A, fil):
    pad = int((fil-1)/2)
    for  i in range(1,pad+1):
        up = np.array(A[2*i-1,:])
        down = np.array(A[-2*i,:])
        Matrix_up_down = np.vstack([up,A,down])
        left = np.array(Matrix_up_down[:,2*i-1]).reshape(len(Matrix_up_down),1)
        right = np.array(Matrix_up_down[:,-2*i]).reshape(len(Matrix_up_down),1)
        Matrix_left_right = np.hstack([left,Matrix_up_down,right,])
        A = Matrix_left_right
    return A

# 参数说明
# n   array size
# fil  filter size5*5
# pad = (fil-1)/2  padding大小


def recreate_whole_data(dim,fil,input_image):
    whole_origin_matrix_array = np.array([split_one_channel(input_image,i) for i in range(1,dim+1)])
    whole_mirror_matrix_array = np.array([mirror_matrix(whole_origin_matrix_array[i],fil) for i in range(dim)])
    return whole_mirror_matrix_array


## 分割patch  work out 一个维度的patch
def create_patch_one(A,fil):
    whole_arr = list()
    row = A.shape[0]-fil+1
    column = A.shape[1]-fil+1
    for i in range(row):
        for j in range(column):
            temp_arr = list()
            for z in range(fil):
                p = A[i+z,j:j+fil]
                temp_arr.append(p)
            one_arr = np.vstack(temp_arr)
            whole_arr.append(one_arr)
    return np.array(whole_arr)
    
def create_patch_whole(A,fil,dim):
    temp = [create_patch_one(split_one_channel(A,i),fil) for i in range(1,dim+1)]
    return np.array(temp)


def create_patch_cube(patch,fil,dim):
    temp = list()
    nums = patch[0].shape[0] # the nums of every dim of windows  n - f + 1
    for j in range(nums):
        temp_re = np.vstack([patch[i][j] for i in range(dim)]).reshape(dim,fil,fil)
        temp_re1= temp_re.transpose(2,1,0) # switch 1,3 dim
        temp_re2 = temp_re1.transpose(1,0,2) # switch 1,2 dim
        temp.append(temp_re2)
    patch_cubes = np.array(temp)
    return patch_cubes

def whole_sample_lable(patch_cubes):
    sample_lable_list = list()
    for i in range(len(patch_cubes)):
            sample_lable_list.append((patch_cubes[i],out_labels[i]))
    whole_sample_lable_array = np.array(sample_lable_list)
    return whole_sample_lable_array

# 提取有用的标签
def useful_sample_lable(patch_cubes):
    sample_lable_list = list()
    for i in range(len(patch_cubes)):
        if out_labels[i] != 0:
            sample_lable_list.append((patch_cubes[i],out_labels[i]))
    sample_lable_array = np.array(sample_lable_list)
    return sample_lable_array

# 存储numpy文件
def save_sample_label(sample_lable_array,path='H:\data\sample_lable_array.numpy'):
    np.save(path,sample_lable_array)
    
# 载入numpy文件
def load_sample_label(name):
    s = np.load(name)
    return s

## 搭建网络

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import SGD

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [90]:
model = Sequential()
# input : 5x5 patch with 30 channels - > (5,5,30)
# layer1 Conv2d  30 x 3 = 90 filters
model.add(Conv2D(90, (3, 3), activation='relu', input_shape=(100, 100, 3)))

model.add(Conv2D(270, (3, 3), activation='relu'))

          
model.add(Dense(180, activation='relu'))
model.add(Dense(16, activation='softmax'))
          
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

## 分类

In [5]:
DIM = 30
FIL = 5

In [6]:
inp = loadmat('F:\data\Indian_pines_corrected.mat')['indian_pines_corrected']
out = loadmat('F:\data\Indian_pines_gt.mat')['indian_pines_gt']

In [7]:
pca_return_array_alpha = pca_hpi(inp,DIM) # pca

[6.84937994e-01 2.35313708e-01 1.49635476e-02 8.21541515e-03
 6.95011265e-03 5.17010703e-03 3.99681431e-03 3.62359687e-03
 3.07127396e-03 2.93211894e-03 2.67352983e-03 2.49230109e-03
 2.24688362e-03 1.89388746e-03 1.69433607e-03 1.56043457e-03
 1.53162474e-03 1.35012945e-03 1.00138897e-03 9.24870614e-04
 8.47876347e-04 7.64378123e-04 6.64576210e-04 6.45652652e-04
 6.16270167e-04 5.60819361e-04 5.43065926e-04 5.14765886e-04
 4.20669358e-04 3.63109771e-04]


In [8]:
pca_return_array_beta = recreate_whole_data(DIM,FIL,pca_return_array_alpha) # padd 并镜像

In [9]:
pca_return_array_beta.shape # 145+2*（f-1）/2

(30, 149, 149)

In [10]:
## 将pca_return beta 变为 149--3  x  149--2  x 30 形状
pca_return_array_beta = pca_return_array_beta.transpose(2,1,0) #switch dim 1,3

In [11]:
pca_return_array_beta.shape

(149, 149, 30)

In [12]:
pca_return_array_beta = pca_return_array_beta.transpose(1,0,2) # switch dim 1,2

In [13]:
pca_return_array_beta.shape

(149, 149, 30)

In [14]:
## ## 将每个 filxfil (5x5)的patch取出来， 并将每个channel的信息放入一个集合
patch_dim = create_patch_whole(pca_return_array_beta,FIL,DIM)

In [15]:
patch_dim.shape  # 30 个dim  every dim 21025个window (sample)    window(5x5)

(30, 21025, 5, 5)

In [16]:
patch_cubes = create_patch_cube(patch_dim,FIL, DIM)

In [17]:
patch_cubes.shape   # 21025个 sample 每个sample是一个patch 30dim  5x5

(21025, 5, 5, 30)

In [18]:
out.shape

(145, 145)

In [19]:
out_labels = out.reshape(145*145,)

In [20]:
out_labels.shape

(21025,)

### 纠错

In [79]:
patch_cubes.shape

(21025, 5, 5, 30)

In [29]:
patch_cubes[0].shape

(5, 5, 30)

In [31]:
patch_cubes[1].shape

(5, 5, 30)

In [32]:
temp = list()
for i in range(patch_cubes.shape[0]):
    temp.append([patch_cubes[i],out_labels[i]])
s =np.array(temp)

In [42]:
s[1][0].shape

(5, 5, 30)

## 纠错过程

In [22]:
sample_lable_array = useful_sample_lable(patch_cubes)

In [45]:
sample_lable_array.shape  # 10249个样本 ，2个values, 1st sample ,  2st lables

(10249, 2)

In [87]:
## 存储sample_label_array文件
save_sample_label(sample_lable_array,'H:\data\sample_lable_array')

In [88]:
s10 = load_sample_label('H:\data\sample_lable_array.npy')

In [89]:
s10.shape #### 搞定

(10249, 2)

In [53]:
def 
temp = list()
for i in range(sample_lable_array.shape[0]):
    temp.append(sample_lable_array[i][0])
np.array(temp).shape

(10249, 5, 5, 30)

In [54]:
temp2 = list()
for i in range(sample_lable_array.shape[0]):
    temp2.append(sample_lable_array[i][1])
np.array(temp2).shape

(10249,)

In [73]:
s4 = sample_lable_array[:,-1] ## 取出标签
s4.shape

(10249,)

In [68]:
s5 = np.array([sample_lable_array[:,0][i]for i in range(sample_lable_array.shape[0])])  ## 取出数据
s5.shape

(10249, 5, 5, 30)

In [69]:
# def scaler_save(sample_lable_arry, path = 'H:\data\cnn_3d_spec_spi.csv'):
#     ## 归一化
#     data_D = np.array([sample_lable_array[:,0][i]for i in range(sample_lable_array.shape[0])])
#     data_L = sample_lable_array[:,-1]

#     ## 存储
#     new = np.column_stack((data_D, data_L))
#     new_ = pd.DataFrame(new)
#     new_.to_csv(path,header=False,index=False)

In [ ]:
# scaler_save(sample_lable_array, path='H:\data\cnn_3d_spec_spi.csv')

In [74]:
data_train, data_test, label_train, label_test = train_test_split(s5,s4,test_size=0.9) ## 分割训练测试集合

In [78]:
data_train.shape #分割标签和样本

(1024, 5, 5, 30)

In [24]:
#### --------------------------------放入云服务器计算-----------------------------------------######
# model.fit(x_train, y_train, batch_size=32, epochs=10)
# score = model.evaluate(x_test, y_test, batch_size=32)